In [1]:
# !pip install pandas

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import datetime as dt
from datetime import timedelta
import json
import os
import csv
import matplotlib.pyplot as plt


In [2]:
localdir=os.getcwd()
# localdir

In [16]:
folder=[folder for folder in os.listdir(localdir) if 'experiments data' in folder][0]
print(folder, os.listdir(folder), len(os.listdir(folder)))

experiments data ['A1_barterGame.json', 'A1_OnlyChat_ChatIncludeWhisper.log', 'A2_barterGame.json', 'A2_ChatIncludeWhisper.log', 'A3_barterGame.json', 'A3_ChatIncludeWhisper.log', 'A4_barterGame.json', 'A4_ChatIncludeWhisper.log', 'A5_barterGame.json', 'A5_ChatIncludeWhisper.log', 'A6_barterGame.json', 'A6_ChatIncludeWhisper.log', 'A7_barterGame.txt', 'A7_ChatIncludeWhisper.log', 'B1_barterGame.json', 'B1_ChatIncludeWhisper.log', 'B2_barterGame.json', 'B2_ChatIncludeWhisper.log', 'B3_barterGame.json', 'B3_ChatIncludeWhisper.log', 'B4_barterGame.json', 'B4_ChatIncludeWhisper.log', 'B5_barterGame.json', 'B5_ChatIncludeWhisper.log', 'B6_barterGame.json', 'B6_ChatIncludeWhisper.log', 'C1_barterGame.json', 'C1_ChatIncludeWhisper.log', 'C2_barterGame.json', 'C2_ChatIncludeWhisper.log', 'C3_barterGame.json', 'C3_ChatIncludeWhisper.log', 'C4_barterGame.json', 'C4_ChatIncludeWhisper.log', 'C5_barterGame.json', 'C5_ChatIncludeWhisper.log', 'C6_barterGame.json', 'C6_ChatIncludeWhisper.log', 'C7_b

#### Get players_list 

In [27]:
json_files = [filename for filename in os.listdir(folder) if '.json' in filename or '.txt' in filename]
# json_files, len(json_files)

In [31]:
ACCEPTED = 'ACCEPTED'
DECLINED = 'DECLINED'
CANCELLED = 'CANCELLED'

game_trades_df_dict = {}
game_final_df_dict = {}
game_list = []
gametype_list = []
winner_list = []
score_list = []
for filename in json_files:
    print(filename)
    gamename = filename.split('_barterGame')[0]
    data=open(os.path.join(localdir,folder,filename), encoding='utf8')
    data_load=json.load(data)
    data_load=data_load['barterGames'][0]
    trades_df=df.from_records(data_load["trades"])
    for ind in trades_df.index:
        trades_df.loc[ind,"beginTimestamp"]=pd.Timestamp(trades_df.loc[ind,"beginTimestamp"])
        trades_df.loc[ind,"endTimestamp"]=pd.Timestamp(trades_df.loc[ind,"endTimestamp"])
    # print(trades_df.shape, filename)
    game_trades_df_dict[filename.split('_')[0]] = trades_df

A1_barterGame.json
A2_barterGame.json
A3_barterGame.json
A4_barterGame.json
A5_barterGame.json
A6_barterGame.json
A7_barterGame.txt
B1_barterGame.json
B2_barterGame.json
B3_barterGame.json
B4_barterGame.json
B5_barterGame.json
B6_barterGame.json
C1_barterGame.json
C2_barterGame.json
C3_barterGame.json
C4_barterGame.json
C5_barterGame.json
C6_barterGame.json
C7_barterGame.txt


In [34]:
chat_files = [filename for filename in os.listdir(folder) if '.log' in filename]
chat_files, len(chat_files)

(['A1_OnlyChat_ChatIncludeWhisper.log',
  'A2_ChatIncludeWhisper.log',
  'A3_ChatIncludeWhisper.log',
  'A4_ChatIncludeWhisper.log',
  'A5_ChatIncludeWhisper.log',
  'A6_ChatIncludeWhisper.log',
  'A7_ChatIncludeWhisper.log',
  'B1_ChatIncludeWhisper.log',
  'B2_ChatIncludeWhisper.log',
  'B3_ChatIncludeWhisper.log',
  'B4_ChatIncludeWhisper.log',
  'B5_ChatIncludeWhisper.log',
  'B6_ChatIncludeWhisper.log',
  'C1_ChatIncludeWhisper.log',
  'C2_ChatIncludeWhisper.log',
  'C3_ChatIncludeWhisper.log',
  'C4_ChatIncludeWhisper.log',
  'C5_ChatIncludeWhisper.log',
  'C6_ChatIncludeWhisper.log',
  'C7_ChatIncludeWhisper.log'],
 20)

In [52]:
BROADCAST = 'broadcast'
WHISPER = 'whisper'

# players_list = ['Alan', 'Kalyani', 'Lamar', 'Sarah', 'Sokka']

game_chat_dict= {}
player_chat_count_dict = {}
player_chat_lines_dict = {}
print(player_chat_count_dict)

required_dict = {'gametype': [], 'game': [], 'player':[], 'mode':[], 'line':[]}
gametype_list = []
game_list = []
player_list = []
mode_list = []
line_list = []

for filename in chat_files:
    total_broadcast = 0
    total_whisper = 0
    print(filename)
    player_set = set(np.unique(game_trades_df_dict[filename.split('_')[0]]['requester'])) | set(np.unique(game_trades_df_dict[filename.split('_')[0]]['requested']))
    print(player_set, len(player_set))
    player_chat_lines_dict[filename] = {player: {BROADCAST: [], WHISPER: []} for player in player_set}
    game_chat_dict[filename]= {WHISPER:[], BROADCAST:[]}
    player_chat_count_dict[filename] = {player: {BROADCAST: 0, WHISPER: 0} for player in player_set}
    chatfile=open(os.path.join(localdir,folder,filename), encoding='utf8')
    lines=chatfile.readlines()
    chatfile.close()
    
    if 'C4_' in filename or 'C5_' in filename:
        # [CHAT] case
        for line in lines:
        # continue
            if '[CHAT] <' in line:
                # broadcast chat
                game_chat_dict[filename][BROADCAST].append(line)
                player_name = line.split('[CHAT] <')[1].split('> ')[0]
                line_ = line.split(player_name+'> ')[1]
                # print(player_name)
                if player_name in player_set:
                    player_chat_count_dict[filename][player_name][BROADCAST] += 1
                    player_chat_lines_dict[filename][player_name][BROADCAST].append(line)
                    total_broadcast += 1
        
                    game_list.append(filename.split('_')[0])
                    if 'A' in filename.split('_')[0]:
                        gametype_list.append('Competitive')
                    elif 'B' in filename.split('_')[0]:
                        gametype_list.append('Baseline')
                    else:
                        gametype_list.append('Cooperative')
                    player_list.append(player_name)
                    mode_list.append(BROADCAST)
                    line_list.append(line_)
            elif '[CHAT] [SS] [' in line:
                # whisper chat
                game_chat_dict[filename][WHISPER].append(line)
                player_name = line.split('[CHAT] [SS] [')[1].split(' -> ')[0]
                if player_name in player_set:
                    # player_name = line.split('INFO]: ')[1].split(' issued server command: /msg')[0]
                    player_chat_count_dict[filename][player_name][WHISPER] += 1
                    player_chat_lines_dict[filename][player_name][WHISPER].append(line)
                    total_whisper += 1
        
                    game_list.append(filename.split('_')[0])
                    if 'A' in filename.split('_')[0]:
                        gametype_list.append('Competitive')
                    elif 'B' in filename.split('_')[0]:
                        gametype_list.append('Baseline')
                    else:
                        gametype_list.append('Cooperative')
                    player_list.append(player_name)
                    mode_list.append(WHISPER)
                    line_ = line.split(' -> ')[1]
                    line_list.append(line_)            
    else:
        # [Not Secure] case
        for line in lines:
            if "[Not Secure] <" in line:
                # broadcast chat
                game_chat_dict[filename][BROADCAST].append(line)
                player_name = line.split('[Not Secure] <')[1].split('> ')[0]
                line_ = line.split(player_name+'> ')[1]
                # print(player_name)
                if player_name in player_set:
                    player_chat_count_dict[filename][player_name][BROADCAST] += 1
                    player_chat_lines_dict[filename][player_name][BROADCAST].append(line)
                    total_broadcast += 1
        
                    game_list.append(filename.split('_')[0])
                    if 'A' in filename.split('_')[0]:
                        gametype_list.append('Competitive')
                    elif 'B' in filename.split('_')[0]:
                        gametype_list.append('Baseline')
                    else:
                        gametype_list.append('Cooperative')
                    player_list.append(player_name)
                    mode_list.append(BROADCAST)
                    line_list.append(line_)
            elif " /msg " in line: #or "/r " in line or "/tell" in line:
                # whisper chat
                game_chat_dict[filename][WHISPER].append(line)
                left_line = line.split(' /msg ')[0]
                player_name = [ply for ply in player_set if ply in left_line]
                if player_name:
                    player_name = player_name[0]                    
                    # player_name = line.split('INFO]: ')[1].split(' issued server command: /msg')[0]
                    player_chat_count_dict[filename][player_name][WHISPER] += 1
                    player_chat_lines_dict[filename][player_name][WHISPER].append(line)
                    total_whisper += 1
        
                    game_list.append(filename.split('_')[0])
                    if 'A' in filename.split('_')[0]:
                        gametype_list.append('Competitive')
                    elif 'B' in filename.split('_')[0]:
                        gametype_list.append('Baseline')
                    else:
                        gametype_list.append('Cooperative')
                    player_list.append(player_name)
                    mode_list.append(WHISPER)
                    line_ = line.split(' /msg ')[1]
                    line_list.append(line_)

    print('total_broadcast = ', total_broadcast)
    print('total_whisper = ', total_whisper)
    dms= len(game_chat_dict[filename][WHISPER])
    print('no.of whisper chats = ', dms)
    globe= len(game_chat_dict[filename][BROADCAST])
    print('no. of broadcasts = ', globe)
    print('total no. of chats = ', dms+globe)
    print('whisper chats (% of total) = ', round(dms/(dms+globe) *100, 2))
    print(player_chat_count_dict[filename])    
    print('##################')

required_dict = {'gametype': gametype_list, 'game': game_list, 'player': player_list,
                 'mode': mode_list, 'line': line_list}
result_df = df.from_dict(required_dict)

{}
A1_OnlyChat_ChatIncludeWhisper.log
{'Zay45', 'ferretium', 'labrat2368', 'labmember009', 'ACrashinTime', 'Captain_India4', 'kalyaniplays'} 7
total_broadcast =  544
total_whisper =  0
no.of whisper chats =  0
no. of broadcasts =  545
total no. of chats =  545
whisper chats (% of total) =  0.0
{'Zay45': {'broadcast': 106, 'whisper': 0}, 'ferretium': {'broadcast': 107, 'whisper': 0}, 'labrat2368': {'broadcast': 47, 'whisper': 0}, 'labmember009': {'broadcast': 67, 'whisper': 0}, 'ACrashinTime': {'broadcast': 70, 'whisper': 0}, 'Captain_India4': {'broadcast': 72, 'whisper': 0}, 'kalyaniplays': {'broadcast': 75, 'whisper': 0}}
##################
A2_ChatIncludeWhisper.log
{'TheSurvivorGuy', 'CJL2213', 'ultimate_glory', 'NerdyPiggy', 'BlackeyeBob345'} 5
total_broadcast =  64
total_whisper =  233
no.of whisper chats =  233
no. of broadcasts =  68
total no. of chats =  301
whisper chats (% of total) =  77.41
{'TheSurvivorGuy': {'broadcast': 13, 'whisper': 65}, 'CJL2213': {'broadcast': 25, 'whi

In [53]:
result_df

,gametype,game,player,mode,line
0,Competitive,A1,ferretium,broadcast,i need leather\n
1,Competitive,A1,kalyaniplays,broadcast,does anyone have white wool\n
2,Competitive,A1,ferretium,broadcast,i do\n
3,Competitive,A1,Captain_India4,broadcast,i have 1\n
4,Competitive,A1,Zay45,broadcast,yeah i got white wool\n
...,...,...,...,...,...
5969,Cooperative,C7,OrganicSugar,broadcast,gg everyone :)\n
5970,Cooperative,C7,Protocol0,broadcast,llama skeletons lol\n
5971,Cooperative,C7,AndrewLukkuz,broadcast,woah wth lol\n
5972,Cooperative,C7,Protocol0,broadcast,um ok\n


In [54]:
result_df[result_df['mode']==WHISPER]

,gametype,game,player,mode,line
544,Competitive,A2,ultimate_glory,whisper,BlackeyeBob345 i cant wait to be so much bette...
545,Competitive,A2,CJL2213,whisper,BlackeyeBob345 :)\n
546,Competitive,A2,CJL2213,whisper,ultimate_glory I see you\n
547,Competitive,A2,BlackeyeBob345,whisper,CJL2213 RAAAAHHHHH\n
548,Competitive,A2,CJL2213,whisper,BlackeyeBob345 give me all your goods or I wil...
...,...,...,...,...,...
5858,Cooperative,C7,Archangel6266,whisper,"OrganicSugar I have it, I'll trade forthe wood!\n"
5861,Cooperative,C7,Archangel6266,whisper,OrganicSugar Thanks!\n
5863,Cooperative,C7,OrganicSugar,whisper,archangel6266 thank you\n
5882,Cooperative,C7,Archangel6266,whisper,Protocol0 I do. I need wood or diamond axes\n


In [55]:
result_df.shape

(5974, 5)

In [56]:
ego_words_list = ['I\'m ', 'I need ', 'I want ', 'I could use ', 'do you have ', 'Do you have ', 'I still need ']

def get_ego_column(r):
    line_ = r['line']
    r['ego'] = 0
    for ele in ego_words_list:
        if ele in line_:
            r['ego'] = 1
            break
    return r


result_df = result_df.apply(get_ego_column, axis=1)

In [57]:
result_df

,gametype,game,player,mode,line,ego
0,Competitive,A1,ferretium,broadcast,i need leather\n,0
1,Competitive,A1,kalyaniplays,broadcast,does anyone have white wool\n,0
2,Competitive,A1,ferretium,broadcast,i do\n,0
3,Competitive,A1,Captain_India4,broadcast,i have 1\n,0
4,Competitive,A1,Zay45,broadcast,yeah i got white wool\n,0
...,...,...,...,...,...,...
5969,Cooperative,C7,OrganicSugar,broadcast,gg everyone :)\n,0
5970,Cooperative,C7,Protocol0,broadcast,llama skeletons lol\n,0
5971,Cooperative,C7,AndrewLukkuz,broadcast,woah wth lol\n,0
5972,Cooperative,C7,Protocol0,broadcast,um ok\n,0


In [58]:
result_df.groupby(by=['game','player', 'mode']).agg({'line':len, 'ego':np.sum})

C:\Users\Vijay Bhaskar\AppData\Local\Temp\ipykernel_21724\2181509277.py:1: FutureWarning: The provided callable <function sum at 0x0000026532366A20> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  result_df.groupby(by=['game','player', 'mode']).agg({'line':len, 'ego':np.sum})


line  ego
game player         mode                
A1   ACrashinTime   broadcast    70    4
     Captain_India4 broadcast    72    0
     Zay45          broadcast   106    0
     ferretium      broadcast   107    7
     kalyaniplays   broadcast    75    6
...                             ...  ...
C7   OrganicSugar   whisper       5    1
     Protocol0      broadcast    78    8
                    whisper       2    0
     firestar5467   broadcast    18    0
                    whisper       9    2

[169 rows x 2 columns]

In [60]:
whisper_ego_df = result_df[result_df['mode']==WHISPER]
ego_whisper_count_df = whisper_ego_df.groupby(by=['game','player']).agg({'line':len, 'ego':np.sum})
ego_whisper_count_df.to_excel('ego-whisper-human-chats-df.xlsx')

broadcast_ego_df = result_df[result_df['mode']==BROADCAST]
ego_broadcast_count_df = broadcast_ego_df.groupby(by=['game','player']).agg({'line':len, 'ego':np.sum})
ego_broadcast_count_df.to_excel('ego-broadcast-human-chats-df.xlsx')

# ego_count_df = result_df.groupby(by=['game','player', 'mode']).agg({'line':len, 'ego':np.sum})
# ego_count_df.to_excel('ego-count-agent-chats-df.xlsx')

C:\Users\Vijay Bhaskar\AppData\Local\Temp\ipykernel_21724\1505562673.py:2: FutureWarning: The provided callable <function sum at 0x0000026532366A20> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ego_whisper_count_df = whisper_ego_df.groupby(by=['game','player']).agg({'line':len, 'ego':np.sum})
C:\Users\Vijay Bhaskar\AppData\Local\Temp\ipykernel_21724\1505562673.py:6: FutureWarning: The provided callable <function sum at 0x0000026532366A20> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ego_broadcast_count_df = broadcast_ego_df.groupby(by=['game','player']).agg({'line':len, 'ego':np.sum})
